In [1]:
import numpy as np
from tqdm import tqdm

In [2]:
Nseq = 100
I = 50 # Sequence length
K = 5 # No. of event types

In [3]:
mu = np.random.rand(K)
alpha = np.random.rand(K, K)
delta = np.random.rand(K, K)*10+10
delta

array([[14.14409235, 12.91547667, 13.96045469, 12.74541805, 11.66034563],
       [12.06929461, 17.88525819, 14.76169548, 14.51562821, 19.35143961],
       [13.37772184, 13.25980361, 12.26912754, 14.72817983, 19.72283304],
       [11.39141622, 12.23652367, 16.74639478, 18.96141384, 18.12468294],
       [15.92639197, 12.57913233, 16.01043102, 15.24965139, 17.79871896]])

In [4]:
tarr = np.zeros((Nseq, I))
evArr = np.zeros((Nseq, I), dtype=int)
for seq in tqdm(range(Nseq), position=0, leave=True): # No. of sequences
    for i in range(I): # No. of timestamps in each sequence
        
        # We have to select the next event
        # First, we find the next time for each event type
        t_i_k = np.zeros(K)
        for k in range(K):
            # Find the intensity of the k^th type at i^th timestamp
            
            C = True
            t = 0. if i==0 else tarr[seq, i-1]
            # Find the supremum
            # iterate through previous time stamps
            lamb_sup_k_i = mu[k]
            for j in range(i):
                lamb_sup_k_i += alpha[k, evArr[seq, j] ]
            while C:                    
                Del = -np.log(np.random.rand())/lamb_sup_k_i
                t += Del
                lamb_k_i = mu[k]
                for j in range(i):
                    lamb_k_i += alpha[k, evArr[seq, j]] * np.exp(-delta[k, evArr[seq, j]]*
                                                                 (t-tarr[seq, j]))
                    
                # Decide to accept the time
                if lamb_k_i/lamb_sup_k_i > np.random.rand():
                    C = False
                    
            t_i_k[k] = t
        
        # Store the earliest event
        tarr[seq, i] = np.min(t_i_k)
        evArr[seq, i] = np.argmin(t_i_k)

100%|██████████| 100/100 [01:11<00:00,  1.41it/s]


In [5]:
t0arr = np.zeros((Nseq, I+1))
t0arr[:, 1:] = tarr

In [6]:
t0arr[1]

array([ 0.        ,  0.24618728,  0.59853352,  0.90344368,  2.0488286 ,
        2.21266063,  2.23426939,  2.24472369,  2.31341376,  2.43084177,
        2.53533944,  2.81105096,  3.24652072,  3.41113015,  4.70811786,
        4.78176352,  5.14897777,  5.52720276,  5.9132132 ,  6.0158646 ,
        6.80453826,  6.89844776,  7.17759905,  8.1436063 ,  8.3469454 ,
        8.65030729,  8.7403492 ,  8.74819479,  8.93232472,  9.287554  ,
        9.53425959, 11.07923162, 11.24242628, 11.47913685, 11.57236845,
       11.65066927, 12.73414155, 12.74958148, 13.41983491, 13.53702436,
       13.55951013, 13.56234879, 13.56899595, 13.635658  , 13.68139901,
       13.83981354, 14.58773325, 15.50278929, 16.2687381 , 16.69719912,
       16.7709882 ])

In [7]:
np.save("Events.npy", evArr)
np.save("Times.npy", t0arr)
np.save("Mu.npy", mu)
np.save("alpha.npy", alpha)
np.save("Delta.npy", delta)

In [8]:
evArr.shape

(100, 50)